In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
import lib
import plotting
import markov_chain as mc
import markov_tools as mt

In [4]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

In [5]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [6]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [7]:
results_prefix = "[WIKIDATA]"

sequence_file = os.path.join(cfg.get("directory", "exchange"), results_prefix + "001_clustering.p")
labels_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_labels.p")

In [8]:
df = pd.read_pickle(sequence_file)
df.head()

,length,sequence,user_id,stat_dist,kmeans_2,kmeans_3,kmeans_4,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10
0,1511,"[REVERT_MANUAL, BREAK, CLAIM_CREATE, BREAK, CL...",1,"[0.022260285362644863, 0.01999793118304165, 0....",1,0,3,2,2,4,1,2,2
1,9,"[ENTITY_CREATE, BREAK, SITELINK_ADD, BREAK, ME...",1000036,"[0.020070153072094386, 0.02007015307209438, 0....",0,0,2,4,0,0,0,0,0
2,7,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",1000078,"[0.01947397679104996, 0.019473976791049966, 0....",0,2,1,1,1,1,5,3,3
3,9,"[ENTITY_CREATE, BREAK, SITELINK_REMOVE, SITELI...",100008,"[0.02042483183496783, 0.020424831834967825, 0....",0,2,1,1,1,1,5,0,0
4,44,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",100012,"[0.01973684210526316, 0.019736842105263157, 0....",0,2,1,1,1,1,5,3,3


In [9]:
labels = pd.read_pickle(labels_file)['label'].values

We test only one clustering (numer of clusters) at a time.  
For each label/action we conduct a pairwise tukey's test to highlight the actions that show significant difference.

Note: as this is a pairwise comparison for each single label this generates a lot of output!

In [10]:
num_clusters = 6
df_selector = "kmeans_{n}".format(n=num_clusters)

In [28]:
for i, l in enumerate(labels):
    vals = df['stat_dist'].apply(lambda x: x[i])
    lbls = df[df_selector]
    comp_object = MultiComparison(vals, lbls)
    hsd = comp_object.tukeyhsd(alpha=0.05)
    print("{l}: {s}".format(l=l, s=sum(hsd.reject)))
    print(hsd.summary())

ALIAS_ADD: 12
Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower   upper  reject
---------------------------------------------
  0      1    -0.0043  -0.0044 -0.0041  True 
  0      2     -0.002  -0.0023 -0.0018  True 
  0      3    -0.0044  -0.0048  -0.004  True 
  0      4     -0.003  -0.0033 -0.0028  True 
  0      5    -0.0043  -0.0045 -0.0041  True 
  1      2     0.0022   0.002   0.0025  True 
  1      3    -0.0001  -0.0005  0.0003 False 
  1      4     0.0012   0.001   0.0015  True 
  1      5      -0.0   -0.0003  0.0002 False 
  2      3    -0.0024  -0.0028  -0.002  True 
  2      4     -0.001  -0.0013 -0.0007  True 
  2      5    -0.0023  -0.0025  -0.002  True 
  3      4     0.0014   0.0009  0.0018  True 
  3      5     0.0001  -0.0003  0.0005 False 
  4      5    -0.0013  -0.0016  -0.001  True 
---------------------------------------------
ALIAS_REMOVE: 10
Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower   upper  